# OpenAI functions

Certain OpenAI models (like gpt-3.5-turbo-0613 and gpt-4-0613) have been fine-tuned to detect when a function should be called and respond with the inputs that should be passed to the function. In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call those functions. The goal of the OpenAI Function APIs is to more reliably return valid and useful function calls than a generic text completion or chat API.

The OpenAI Functions Agent is designed to work with these models.

Install `openai`, `google-search-results` packages which are required as the LangChain packages call them internally.

In [ ]:
! pip install openai tavily-python

## Initialize tools

We will first create some tools we can use

In [2]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub

In [24]:
tools = [TavilySearchResults(max_results=1)]

## Create Agent

In [28]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [29]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [30]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)

## Run the Agent

In [26]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://www.ibm.com/topics/langchain', 'content': "LangChain is essentially a library of abstractions for Python and Javascript, representing common steps and concepts  LangChain is an open source orchestration framework for the development of applications using large language models  other LangChain features, like the eponymous chains.  LangChain provides integrations for over 25 different embedding methods, as well as for over 50 different vector storesLangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Available in both Python- and Javascript-based libraries, LangChain's tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents . LangChain serves as a generic interface for ..."}]LangChain is an open-source orchestration framework for the developm

{'input': 'what is LangChain?',
 'output': 'LangChain is an open-source orchestration framework for the development of applications using large language models (LLMs). It provides integrations for over 25 different embedding methods and over 50 different vector stores. LangChain simplifies the process of building LLM-driven applications such as chatbots and virtual agents. It is available in both Python- and Javascript-based libraries. You can find more information about LangChain on the IBM website: [LangChain - IBM](https://www.ibm.com/topics/langchain)'}

## Using LCEL

We will first use LangChain Expression Language to create this agent

In [31]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [32]:
from langchain.tools.render import format_tool_to_openai_function

In [33]:
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [34]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [35]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [36]:
from langchain.agents import AgentExecutor

In [37]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://www.ibm.com/topics/langchain', 'content': "LangChain is essentially a library of abstractions for Python and Javascript, representing common steps and concepts  LangChain is an open source orchestration framework for the development of applications using large language models  other LangChain features, like the eponymous chains.  LangChain provides integrations for over 25 different embedding methods, as well as for over 50 different vector storesLangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Available in both Python- and Javascript-based libraries, LangChain's tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents . LangChain serves as a generic interface for ..."}]LangChain is an open source orchestration framework for the developm

{'input': 'what is LangChain?',
 'output': 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs). It is available in both Python- and Javascript-based libraries and provides tools and APIs to simplify the process of building LLM-driven applications such as chatbots and virtual agents. LangChain serves as a generic interface for integrating over 25 different embedding methods and over 50 different vector stores. You can find more information about LangChain on the IBM website [here](https://www.ibm.com/topics/langchain).'}